Imports

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks'

!pip install import-ipynb

import import_ipynb

from bce_loss_functions import *

from cce_loss_functions import *

from custom_metrics import *

from hmda_data_loader import *

import pandas as pd

import numpy as np

import random as rd

import tensorflow as tf

from tensorflow.python.framework import constant_op

from tensorflow import keras

from keras import layers

from scipy.stats import chi2_contingency

Constants and Variables

In [ ]:
DIRECTORIES = {
    'MODELS':'/content/drive/MyDrive/Colab Notebooks/models/hmda/',
    'OUTPUT':'/content/drive/MyDrive/Colab Notebooks/ouput/hmda/'
}


BASE_RACE_MODEL_DIRS = {
            'HMDA_RACE_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/race/v1/b/',
            'HMDA_RACE_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/race/v2/b/',
}


CAT_RACE_MODEL_DIRS = {
            'HMDA_RACE_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/race/v1/c/',
            'HMDA_RACE_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/race/v2/c/',
}

BASE_SEX_MODEL_DIRS = {
            'HMDA_SEX_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/sex/v1/b/',
            'HMDA_SEX_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/sex/v2/b/',

}

CAT_SEX_MODEL_DIRS = {
            'HMDA_SEX_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/sex/v1/c/',
            'HMDA_SEX_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/sex/v2/c/',
}

BASE_SIMU_MODEL_DIRS = {
            'HMDA_SIMU_B_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/balanced/v1/b/',
            'HMDA_SIMU_B_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/balanced/v2/b/',
            'HMDA_SIMU_R_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/random/v1/b/',
            'HMDA_SIMU_R_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/random/v2/b/',
}

CAT_SIMU_MODEL_DIRS = {
            'HMDA_SIMU_B_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/balanced/v1/c/',
            'HMDA_SIMU_B_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/balanced/v2/c/',
            'HMDA_SIMU_R_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/random/v1/c/',
            'HMDA_SIMU_R_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/simu/random/v2/c/',
}


BASE_RACE_SEX_MODEL_DIRS = {
            'HMDA_RS_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/rs/v1/b/',
            'HMDA_RS_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/rs/v2/b/',
}

CAT_RACE_SEX_MODEL_DIRS = {
            'HMDA_RS_V1':'/content/drive/MyDrive/Colab Notebooks/models/hmda/rs/v1/c/',
            'HMDA_RS_V2':'/content/drive/MyDrive/Colab Notebooks/models/hmda/rs/v2/c/',
}

MODEL_NAMES = {
            'ALPHA_0':'M0.keras',
            'ALPHA_0.25':'M25.keras',
            'ALPHA_0.5':'M5.keras',
            'ALPHA_0.75':'M75.keras'
}

get data

In [ ]:
category = 'race'

print('\nloading test data\n')

test_X,test_Y,test_ = get_data(dataset_type='test',category=category,sample=False,sample_method='random',normalization='std')

test_data = tf.convert_to_tensor(test_X,dtype='float32')

test_targets = tf.convert_to_tensor(test_Y,dtype='float32')

test_Xc = test_[0]

test_data_c = tf.convert_to_tensor(test_Xc,dtype='float32')

test_cats = test_[2]

cat_names = test_[3]

print('test accept:\t\t',int(np.sum(test_[1])))
print('test denied:\t\t',len(test_[1]) - int(np.sum(test_[1])))
print('test accept rate:\t',np.sum(test_[1])/len(test_[1]))

print(test_data.shape)
print(test_targets.shape)
print(test_data_c.shape)



test model functions

In [ ]:
def metrics(y_true,y_pred,f_loss,loss_ver,fair_type,data_type,cat_name,alpha,write=False,writer=None):

    mets = {
        'bac':tf.keras.metrics.BinaryAccuracy(),
        'bce':tf.keras.losses.BinaryCrossentropy(),
        'tp':tf.keras.metrics.TruePositives(),
        'tn':tf.keras.metrics.TrueNegatives(),
        'fp':tf.keras.metrics.FalsePositives(),
        'fn':tf.keras.metrics.FalseNegatives()
    }

    for n,m in mets.items():

        if n == 'bce':
            continue

        m.update_state(y_true,y_pred)

    tar = (mets['tp'].result().numpy() + mets['fn'].result().numpy()) / y_true.shape[0]

    par = (mets['tp'].result().numpy() + mets['fp'].result().numpy()) / y_true.shape[0]

    if write:

        writer.write('{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(loss_ver,
                                                                   fair_type,
                                                                   data_type,
                                                                   cat_name,
                                                                   alpha,
                                                                   mets['bac'].result().numpy(),
                                                                   f_loss,
                                                                   mets['bce'](y_true,y_pred).numpy(),
                                                                   par,
                                                                   int(mets['tp'].result().numpy()),
                                                                   int(mets['tn'].result().numpy()),
                                                                   int(mets['fp'].result().numpy()),
                                                                   int(mets['fn'].result().numpy())))


    return tar,par

def all_metrics(y_true,y_pred,y_pred_tensor,loss_ver,fair_type,data_type,alpha,write=False,writer=None):

    fbce = MaxGroupRiskBCEv1(alpha=alpha) if loss_ver == 'v1' else MaxGroupRiskBCEv2(alpha=alpha)

    yT = tf.gather(y_true,[0],axis=1)

    metrics(yT,y_pred,fbce(y_true,y_pred_tensor).numpy(),loss_ver,fair_type,data_type,'all',alpha,write,writer)

    n_cats = y_true.get_shape()[1] - 1

    tars, pars = [],[]

    i = 0

    for k,v in cat_names.items():

        cat = tf.gather(y_true,[i+1],axis=1)

        n_cat = tf.reduce_sum(cat)

        cYT = tf.reshape(tf.gather(yT,tf.gather(tf.where(cat),[0],axis=1)),[n_cat,1])

        cYP = tf.reshape(tf.gather(y_pred,tf.gather(tf.where(cat),[0],axis=1)),[n_cat,1])

        tar,par = metrics(cYT,cYP,0,loss_ver,fair_type,data_type,v,alpha,write,writer)

        i += 1

def test_model(model,data,targets,cats,loss_ver,fair_type,data_type,alpha,write=False,writer=None):

    y_pred = model.predict(data)

    all_metrics(targets,y_pred,tf.convert_to_tensor(y_pred),loss_ver,fair_type,data_type,alpha,write,writer)

    y_predl = [1 if y > 0.5 else 0 for y in list(y_pred)]

    y_preds = pd.Series(y_predl,name='predictions')

    catss = pd.Series(cats,name='categories')

    contigency = pd.crosstab(index=catss, columns=y_preds)

    print('\n\n',contigency)

    contigency_pct = pd.crosstab(catss, y_preds, normalize='index')

    print('\n',contigency_pct)

    c, p, dof, expected = chi2_contingency(contigency,correction=False)

    return p

def evaluate_model(model_path,loss,loss_name,loss_ver,fair_type,data_type,alpha,eval_data,eval_targets,eval_cats,test_data,test_targets,test_cats,batch_size=256,writer=None):

    model = tf.keras.models.load_model(model_path, custom_objects={loss_name:loss,'CBA':CBA(),'CTP':CTP(),'CTN':CTN(),'CFP':CFP(),'CFN':CFN()})

    results = model.evaluate(eval_data,eval_targets,batch_size=256)

    print('eval loss, eval acc:', results)

    test_model(model,eval_data,eval_targets,eval_cats,loss_ver,fair_type,data_type,alpha,True,writer)

    return test_model(model,test_data,test_targets,test_cats,loss_ver,fair_type,data_type,alpha)


test models

In [ ]:
base_dir = BASE_SEX_MODEL_DIRS if category == 'sex' else BASE_RACE_MODEL_DIRS if category == 'race' else BASE_SIMU_MODEL_DIRS if category == 'simu' else BASE_RACE_SEX_MODEL_DIRS

cat_dir = CAT_SEX_MODEL_DIRS if category == 'sex' else CAT_RACE_MODEL_DIRS if category == 'race' else CAT_SIMU_MODEL_DIRS if category == 'simu' else CAT_RACE_SEX_MODEL_DIRS

fn = DIRECTORIES['OUTPUT'] + 'hmda_{}_p_values.csv'.format(category)

f = open(fn, 'w')

f.write('loss_function_version,sample_method,data_type,alpha_0,alpha_0.25,alpha_0.50,alpha_0.75\n')

fn2 = DIRECTORIES['OUTPUT'] + 'hmda_{}_data.csv'.format(category)

f2 = open(fn2, 'w')

f2.write('loss_function_version,sample_method,data_type,category,alpha_value,bin_accuracy,fbce_loss,bce_loss,approval_rate,true_positives,true_negatives,false_positives,false_negatives\n')

for n,d in base_dir.items():

    print('\n\nmodel set: ',n)

    bp_vals,cp_vals,s = 'base_features,','category_features,',''

    v2 = False

    for a,mn in MODEL_NAMES.items():

        alpha = float(a.split('_')[1])

        dirl = d.split('/')

        loss = MaxGroupRiskBCEv1(alpha=alpha) if 'v1' in dirl else MaxGroupRiskBCEv2(alpha=alpha)

        loss_name = 'v1:MGR_Equity_BCE' if 'v1' in dirl else 'v2:MGR_Equality_BCE'

        loss_ver = 'v1' if 'v1' in dirl else 'v2'

        sample_method = 'class_balanced' if 'balanced' in dirl else 'random'

        print('\n\nalpha value: ',alpha)

        print('\n\nbase feature model')

        p_value = evaluate_model(d+mn,loss,loss_name,loss_ver,sample_method,'base_features',alpha,test_data,test_targets,test_cats,test_data,test_targets,test_cats,writer=f2)

        bp_vals += '{},'.format(p_value)

        print('\n\ncategory feature model')

        p_value = evaluate_model(cat_dir[n]+mn,loss,loss_name,loss_ver,sample_method,'category_features',alpha,test_data_c,test_targets,test_cats,test_data_c,test_targets,test_cats,writer=f2)

        cp_vals += '{},'.format(p_value)

        s = '{},{},'.format(loss_ver,sample_method)

    f.write(s + bp_vals + '\n')
    f.write(s + cp_vals + '\n')

f.close()
f2.close()